# RITMO REPORT

### 1. Sales - by City

In [0]:
sales_by_city = """
WITH BASE_DATA AS (
    SELECT 
        2025_calendar_fcst.city, 2025_calendar_fcst.day, 2025_calendar_fcst.`Final Day` weekday, daily_weight_fcts.comset, budget_n.value Nights_Sold, budget_r.value Revenue_Sold
        
    FROM silver.finance.2025_calendar_fcst 
        LEFT JOIN silver.finance.daily_weight_fcts ON daily_weight_fcts.city = 2025_calendar_fcst.city AND daily_weight_fcts.day = 2025_calendar_fcst.`Final Day`
        LEFT JOIN (SELECT account, city, db_month month, volume value FROM silver.finance.raw_working_file_volume_updates WHERE account = 'Nights Sold') as budget_n ON 2025_calendar_fcst.city = budget_n.city AND budget_n.month = trunc(2025_calendar_fcst.day, 'month')
        LEFT JOIN (SELECT account, city, db_month month, amount value FROM silver.finance.best_case_budget_finance WHERE account = 'Revenue - Sold') as budget_r ON 2025_calendar_fcst.city = budget_r.city AND budget_r.month = trunc(2025_calendar_fcst.day, 'month')
    )

    ,WORKING_DAYS AS (
        SELECT SUM(comset) working_days, city, trunc(day, 'month') month
        FROM BASE_DATA
        GROUP BY city, trunc(day, 'month')
    ) 

    ,FORECASTS AS (
    SELECT
        BASE_DATA.city, BASE_DATA.day, BASE_DATA.weekday, BASE_DATA.comset, 
        ROUND((Revenue_Sold / working_days) * comset) Forecast_r,
        ROUND((Nights_Sold / working_days) * comset) Forecast_n

    FROM BASE_DATA LEFT JOIN WORKING_DAYS ON BASE_DATA.city = WORKING_DAYS.city AND trunc(BASE_DATA.day, 'month') = WORKING_DAYS.month

    WHERE trunc(BASE_DATA.day, 'month') = trunc(NOW(), 'month')
    )

    ,BOOKINGS_DETAILS AS (
        SELECT city, nights, CAST(sell_date AS Date) sell_date, contract_id, type FROM silver.finance.new_sales_and_source
    )

    ,REVENUE_GROUPING AS (
        SELECT sell_date, city, codename, penalty revenue, month, booking_id, 'Penalty' tipo, contract_id FROM silver.finance.penalty_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, Utilities revenue, month, booking_id, 'Utilities' tipo, contract_id FROM silver.finance.utilities_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, service_charge revenue, month, booking_id, 'Service Charge' tipo, contract_id FROM silver.finance.sc_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, rent revenue, month, booking_id, 'Rent' tipo, contract_id FROM silver.finance.rent_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, final_cleaning revenue, month, booking_id, 'Final Cleaning' tipo, contract_id FROM silver.finance.cleaning_per_recognition
    )

    ,REVENUE_PER_RECOGNITION AS (
        SELECT 
        CAST(R.sell_date AS DATE) sell_date, R.city, R.codename, R.revenue, R.month, R.booking_id, R.tipo, R.contract_id, BOOKINGS_DETAILS.type 
        
        FROM REVENUE_GROUPING R LEFT JOIN BOOKINGS_DETAILS ON R.contract_id = BOOKINGS_DETAILS.contract_id
    )

    ,FINAL_TABLE_REVENUE AS (
    SELECT 
        FORECASTS.city, FORECASTS.day, FORECASTS.weekday, FORECASTS.comset weight, forecast_n,  
        CASE WHEN SUM(revenue) IS NULL THEN 0 ELSE SUM(revenue) END AS revenue_sold, FORECASTS.forecast_r revenue_forecast, 
        (CASE WHEN SUM(revenue) IS NULL THEN 0 ELSE SUM(revenue) END) - FORECASTS.forecast_r Revenue_Delta 
    
    FROM FORECASTS 
        LEFT JOIN REVENUE_PER_RECOGNITION ON FORECASTS.city = REVENUE_PER_RECOGNITION.city AND FORECASTS.day = REVENUE_PER_RECOGNITION.sell_date
    
    GROUP BY FORECASTS.city, FORECASTS.day, FORECASTS.weekday, FORECASTS.comset, FORECASTS.Forecast_r, FORECASTS.Forecast_n
    )

    ,FINAL_TABLE_NIGHTS AS (
    SELECT
        FINAL_TABLE_REVENUE.*,
        CASE WHEN SUM(nights) IS NULL THEN 0 ELSE SUM(nights) END AS nights_sold, FINAL_TABLE_REVENUE.forecast_n nights_forecast, 
        (CASE WHEN SUM(nights) IS NULL THEN 0 ELSE SUM(nights) END) - FINAL_TABLE_REVENUE.forecast_n Nights_Delta

    FROM FINAL_TABLE_REVENUE
        LEFT JOIN BOOKINGS_DETAILS ON FINAL_TABLE_REVENUE.city = BOOKINGS_DETAILS.city AND FINAL_TABLE_REVENUE.day = BOOKINGS_DETAILS.sell_date
    
    GROUP BY FINAL_TABLE_REVENUE.city, FINAL_TABLE_REVENUE.day, FINAL_TABLE_REVENUE.weekday, FINAL_TABLE_REVENUE.weight, FINAL_TABLE_REVENUE.revenue_forecast, FINAL_TABLE_REVENUE.Forecast_n,
    revenue_sold, Revenue_Delta
    )

    ,FINAL_TABLE AS (
    SELECT
        city, day, weekday, weight, 
        ROUND(revenue_sold, 2) revenue_sold, ROUND(revenue_forecast, 2) revenue_forecast, ROUND(revenue_delta, 2) revenue_delta,
        ROUND(nights_sold, 2) nights_sold, ROUND(nights_forecast, 2) nights_forecast, ROUND(nights_delta, 2) nights_delta,
        ROUND((revenue_sold / (CASE WHEN nights_sold = 0 THEN 1 ELSE nights_sold END)), 2) Price_Per_Night_Sold,
        ROUND((revenue_forecast / nights_forecast), 2) Price_Per_Night_Forecast,
        ROUND(((revenue_sold / (CASE WHEN nights_sold = 0 THEN 1 ELSE nights_sold END)) - (revenue_forecast / nights_forecast)), 2) Price_Per_Night_Delta

    FROM FINAL_TABLE_NIGHTS

    ORDER BY CASE WHEN city = 'Barcelona' THEN 1
                    WHEN city = 'Madrid' THEN 2
                    WHEN city = 'Lisbon' THEN 3
                    WHEN city = 'Paris' THEN 4
                    WHEN city = 'Berlin' THEN 5
            END ASC, day ASC
    )

    SELECT * FROM FINAL_TABLE
"""

spark.sql(sales_by_city).write.mode("overwrite").saveAsTable("silver.finance.sales_by_city")

### 2. Occupation - by City

In [0]:
occupation_by_city = """
WITH BASE_DATA AS (
        SELECT 
            2025_calendar_fcst.city, 2025_calendar_fcst.day, 2025_calendar_fcst.`Final Day` weekday, daily_weight_fcts.comset, 
            budget_o.value occupied_nights, budget_b.value bookable_nights, budget_n.value ritmo_sold_nights,
            budget_cb.value current_bookable_nights, budget_os.value occupancy_start_month
        
        FROM silver.finance.2025_calendar_fcst 
            LEFT JOIN silver.finance.daily_weight_fcts ON daily_weight_fcts.city = 2025_calendar_fcst.city AND daily_weight_fcts.day = 2025_calendar_fcst.`Final Day`
            LEFT JOIN (SELECT account, city, db_month month, volume value FROM silver.finance.raw_working_file_volume_updates WHERE account = 'Occupied Nights') as budget_o ON 2025_calendar_fcst.city = budget_o.city AND budget_o.month = trunc(2025_calendar_fcst.day, 'month')
            LEFT JOIN (SELECT account, city, db_month month, volume value FROM silver.finance.raw_working_file_volume_updates WHERE account = '# Bookable') as budget_b ON 2025_calendar_fcst.city = budget_b.city AND budget_b.month = trunc(2025_calendar_fcst.day, 'month')
            LEFT JOIN (SELECT account, city, db_month month, volume value FROM silver.finance.raw_working_file_volume_updates WHERE account = '# Bookable in Current Month') as budget_cb ON 2025_calendar_fcst.city = budget_cb.city AND budget_cb.month = trunc(2025_calendar_fcst.day, 'month')
            LEFT JOIN (SELECT account, city, db_month month, volume value FROM silver.finance.raw_working_file_volume_updates WHERE account = '#�Nights Sold Ritmo') as budget_n ON 2025_calendar_fcst.city = budget_n.city AND budget_n.month = trunc(2025_calendar_fcst.day, 'month')
            LEFT JOIN (SELECT account, city, db_month month, volume value FROM silver.finance.raw_working_file_volume_updates WHERE account = '# Occupancy Start Month') as budget_os ON 2025_calendar_fcst.city = budget_os.city AND budget_os.month = trunc(2025_calendar_fcst.day, 'month')
    )

    ,WORKING_DAYS AS (
        SELECT SUM(comset) working_days, city, trunc(day, 'month') month
        FROM BASE_DATA
        GROUP BY city, trunc(day, 'month')
    )

    ,WEIGHTS AS (
        SELECT city, db_month month, volume value FROM silver.finance.raw_working_file_volume_updates WHERE account = '# Sales Weight' 
    )

    ,OFFSETS AS (
        SELECT 0 AS offset UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3
    )

    ,FORECASTS AS (
        SELECT 
            BD.city, BD.day, BD.weekday, BD.comset,
            TRUNC(NOW(), 'month') + offset * INTERVAL 1 MONTH AS month_date,
            (((BD.ritmo_sold_nights / WD.working_days) * BD.comset) * w.value) AS Forecast_o,
            BD2.current_bookable_nights Forecast_b

        FROM BASE_DATA BD
            JOIN WORKING_DAYS WD ON BD.city = WD.city AND TRUNC(BD.day, 'month') = WD.month
            JOIN OFFSETS o ON 1 = 1 
            LEFT JOIN WEIGHTS w ON BD.city = w.city AND w.month = TRUNC(NOW(), 'month') + offset * INTERVAL 1 MONTH
            LEFT JOIN (SELECT AVG(current_bookable_nights) current_bookable_nights, city, trunc(day, 'month') day
                    FROM BASE_DATA WHERE current_bookable_nights IS NOT NULL 
                    GROUP BY city, trunc(day, 'month')) BD2 ON BD2.city = BD.city AND BD2.day = TRUNC(NOW(), 'month') + offset * INTERVAL 1 MONTH

        WHERE TRUNC(BD.day, 'month') = TRUNC(NOW(), 'month') AND w.value IS NOT NULL

        ORDER BY BD.city ASC, month_date ASC, BD.day ASC
    )

    ,BOOKINGS_DETAILS AS (
        SELECT city, occupied_nights, check_in, check_out, CAST(sell_date AS Date) sell_date, contract_id, type, month_date FROM silver.finance.occupied_nights_per_booking
    )

    ,VOLUME AS (
        SELECT city, SUM(sales_bookable) bookable, month_date FROM silver.finance.volume_database GROUP BY city, month_date
    )

    ,ACTUALS_BOOKED_PER_SELL_DATE AS (
        SELECT
            city, month_date,
            CASE 
                WHEN date_trunc('month', sell_date) = date_trunc('month', NOW()) THEN CAST(date_trunc('day', sell_date) AS DATE)
                WHEN date_trunc('month', sell_date) < date_trunc('month', NOW()) THEN CAST(date_trunc('month', NOW()) - INTERVAL '1 DAY' AS DATE)
            END AS day,
            SUM(occupied_nights) AS occupied_nights

        FROM BOOKINGS_DETAILS
        
        WHERE date_trunc('month', sell_date) <= date_trunc('month', NOW())
            AND month_date BETWEEN date_trunc('month', NOW())
            AND date_trunc('month', NOW()) + INTERVAL '3 month'

        GROUP BY city, month_date, CASE WHEN date_trunc('month', sell_date) = date_trunc('month', NOW()) THEN CAST(date_trunc('day', sell_date) AS DATE)
                                        WHEN date_trunc('month', sell_date) < date_trunc('month', NOW()) THEN CAST(date_trunc('month', NOW()) - INTERVAL '1 DAY' AS DATE)
                                END
        ORDER BY city ASC, month_date ASC, day ASC
    )

    ,ACTUALS_BOOKABLE_PER_SELL_DATE AS (
        SELECT
            city, month_date, SUM(bookable) bookable
        
        FROM VOLUME WHERE month_date BETWEEN date_trunc('month', NOW()) AND date_trunc('month', NOW()) + INTERVAL '3 month' GROUP BY city, month_date
    )

    ,FORECAST_BOOKED_BEFORE AS (
        SELECT FORECASTS.city, FORECASTS.month_date, occupancy_start_month - SUM(forecast_o) forecast_o, CAST(TRUNC(NOW(), 'month') - INTERVAL 1 DAY AS DATE) day
        FROM FORECASTS
            JOIN (SELECT AVG(occupancy_start_month) occupancy_start_month, city, TRUNC(day, 'month') month_date
                FROM BASE_DATA GROUP BY city, TRUNC(day, 'month')) BD ON BD.city = FORECASTS.city AND BD.month_date = FORECASTS.month_date

        GROUP BY FORECASTS.month_date, FORECASTS.city, occupancy_start_month ORDER BY FORECASTS.city ASC, FORECASTS.month_date ASC
    )

    ,PRE_FORECAST AS (
        SELECT city, day, month_date, weekday, comset, forecast_o, forecast_b FROM FORECASTS
        UNION ALL
        SELECT city, day, month_date, NULL weekday, NULL comset, forecast_o, NULL forecast_b FROM FORECAST_BOOKED_BEFORE
        ORDER BY city ASC, month_date ASC, day ASC
    )

    ,PRE_FINAL AS (
        SELECT 
            F.city, F.month_date, F.day, F.comset, 
            COALESCE(AB.occupied_nights,0) occupied_nights, 
            COALESCE(F.forecast_o,0) forecast_o,
            COALESCE(AB2.bookable, 0) bookable_nights, COALESCE(F2.forecast_b,0) forecast_b
            
        FROM PRE_FORECAST F
            LEFT JOIN (SELECT city, month_date, day, occupied_nights FROM ACTUALS_BOOKED_PER_SELL_DATE) AB ON F.city = AB.city AND F.month_date = AB.month_date AND F.day = AB.day
            LEFT JOIN (SELECT city, month_date, AVG(forecast_b) forecast_b FROM FORECASTS GROUP BY city, month_date) F2 ON F2.city = F.city AND F2.month_date = F.month_date
            LEFT JOIN (SELECT city, month_date, AVG(bookable) bookable FROM ACTUALS_BOOKABLE_PER_SELL_DATE GROUP BY city, month_date) AB2 ON AB2.city = F.city AND AB2.month_date = F.month_date

        ORDER BY AB.city ASC, AB.month_date ASC, AB.day ASC
    )

    ,PRE_DELTAS AS (
        SELECT 
            city, month_date, day, comset, 
            SUM(occupied_nights) OVER (PARTITION BY city, month_date ORDER BY month_date ASC, day ASC) / bookable_nights occupancy,
            SUM(forecast_o) OVER (PARTITION BY city, month_date ORDER BY month_date ASC, day ASC) / forecast_b forecast_occupancy,
            occupied_nights, forecast_o, bookable_nights, forecast_b

        FROM PRE_FINAL
    )

    SELECT 
    city, month_date, day, comset, 
    occupancy, forecast_occupancy, (occupancy - forecast_occupancy) occupancy_delta, 
    occupied_nights, forecast_o, (occupied_nights - forecast_o) / forecast_o ocupied_delta, 
    bookable_nights, forecast_b, (bookable_nights - forecast_b) / forecast_b bookable_delta

    FROM PRE_DELTAS
"""

spark.sql(occupation_by_city).write.mode("overwrite").saveAsTable("silver.finance.occupation_by_city")

### 3. Revenue - by City

In [0]:
revenue_by_city = """
WITH BASE_DATA AS (
        SELECT 
            2025_calendar_fcst.city, 2025_calendar_fcst.day, 2025_calendar_fcst.`Final Day` weekday, daily_weight_fcts.comset, 
            budget_r.value Revenue_Start, budget_y.value Revenue_During
        
        FROM silver.finance.2025_calendar_fcst 
            LEFT JOIN silver.finance.daily_weight_fcts ON daily_weight_fcts.city = 2025_calendar_fcst.city AND daily_weight_fcts.day = 2025_calendar_fcst.`Final Day`
            LEFT JOIN (SELECT city, db_month month, amount value 
                    FROM silver.finance.best_case_budget_finance 
                    WHERE account = 'Revenue Start Month') AS budget_r ON 2025_calendar_fcst.city = budget_r.city AND budget_r.month = trunc(2025_calendar_fcst.day, 'month')
            LEFT JOIN (SELECT city, db_month month, amount value 
                    FROM silver.finance.best_case_budget_finance 
                    WHERE account = 'Revenue Month') AS budget_y ON 2025_calendar_fcst.city = budget_y.city AND budget_y.month = trunc(2025_calendar_fcst.day, 'month')
    )

    ,WORKING_DAYS AS (
        SELECT SUM(comset) working_days, city, trunc(day, 'month') month
        FROM BASE_DATA
        GROUP BY city, trunc(day, 'month')
    ) 

    ,WEIGHTS AS (
        SELECT city, db_month month, volume value FROM silver.finance.raw_working_file_volume_updates WHERE account = '# Sales Weight' 
    )

    ,OFFSETS AS (
        SELECT 0 AS offset UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3
    )

    ,FORECASTS AS (
        SELECT 
            BD.city, BD.day, BD.weekday, BD.comset,
            TRUNC(NOW(), 'month') + offset * INTERVAL 1 MONTH AS month_date,
            ((BD.Revenue_During / WD.working_days) * BD.comset) AS Forecast_r,
            BD.Revenue_During, WD.working_days

        FROM BASE_DATA BD
            JOIN WORKING_DAYS WD ON BD.city = WD.city AND TRUNC(BD.day, 'month') = WD.month
            JOIN OFFSETS o ON 1 = 1 
            LEFT JOIN WEIGHTS w ON BD.city = w.city AND w.month = TRUNC(NOW(), 'month') + offset * INTERVAL 1 MONTH
            LEFT JOIN (SELECT AVG(Revenue_During) current_bookable_nights, city, trunc(day, 'month') day
                    FROM BASE_DATA WHERE Revenue_During IS NOT NULL 
                    GROUP BY city, trunc(day, 'month')) BD2 ON BD2.city = BD.city AND BD2.day = TRUNC(NOW(), 'month') + offset * INTERVAL 1 MONTH

        WHERE TRUNC(BD.day, 'month') = TRUNC(NOW(), 'month') AND w.value IS NOT NULL
                
            UNION ALL

        SELECT DISTINCT city, CAST(trunc(NOW(), 'month') - INTERVAL 1 DAY AS DATE) day, NULL weekday, NULL comset, 
        TRUNC(NOW(), 'month') + offset * INTERVAL 1 MONTH AS month_date, NULL Forecast_r, NULL Revenue_During, NULL working_days
        FROM BASE_DATA JOIN offsets ON 1 = 1
        ORDER BY city DESC, month_date desc
    )

    ,REVENUE_GROUPING AS (
        SELECT sell_date, city, codename, penalty revenue, month, booking_id, 'Penalty' tipo, contract_id FROM silver.finance.penalty_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, Utilities revenue, month, booking_id, 'Utilities' tipo, contract_id FROM silver.finance.utilities_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, service_charge revenue, month, booking_id, 'Service Charge' tipo, contract_id FROM silver.finance.sc_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, rent revenue, month, booking_id, 'Rent' tipo, contract_id FROM silver.finance.rent_per_recognition
        UNION ALL
        SELECT sell_date, city, codename, final_cleaning revenue, month, booking_id, 'Final Cleaning' tipo, contract_id FROM silver.finance.cleaning_per_recognition
    )

    ,REVENUE_PER_RECOGNITION AS (
        SELECT 
        CAST(sell_date AS DATE) sell_date, city, SUM(revenue) revenue, month month_date, contract_id
        
        FROM REVENUE_GROUPING GROUP BY CAST(sell_date AS DATE), city, month, contract_id
    )

    ,ACTUALS_REVENUE_PER_SELL_DATE AS (
        SELECT
            city, month_date,
            CASE 
                WHEN date_trunc('month', sell_date) = date_trunc('month', NOW()) THEN CAST(date_trunc('day', sell_date) AS DATE)
                WHEN date_trunc('month', sell_date) < date_trunc('month', NOW()) THEN CAST(date_trunc('month', NOW()) - INTERVAL '1 DAY' AS DATE)
            END AS day,
            SUM(revenue) AS revenue

        FROM REVENUE_PER_RECOGNITION
        
        WHERE date_trunc('month', sell_date) <= date_trunc('month', NOW())
            AND month_date BETWEEN date_trunc('month', NOW())
            AND date_trunc('month', NOW()) + INTERVAL '3 month'

        GROUP BY city, month_date, CASE WHEN date_trunc('month', sell_date) = date_trunc('month', NOW()) THEN CAST(date_trunc('day', sell_date) AS DATE)
                                        WHEN date_trunc('month', sell_date) < date_trunc('month', NOW()) THEN CAST(date_trunc('month', NOW()) - INTERVAL '1 DAY' AS DATE)
                                END
    )

    ,PRE_FINAL AS (
        SELECT 
            F1.city, F1.month_date, F1.day, F1.comset, 
            AR.revenue, COALESCE(F.forecast_r, F2.forecast_r) forecast_r
            
        FROM FORECASTS F1
            LEFT JOIN ACTUALS_REVENUE_PER_SELL_DATE AR ON AR.day = F1.day AND F1.city = AR.city AND F1.month_date = AR.month_date
            LEFT JOIN (SELECT city, month_date, day, forecast_r, comset FROM FORECASTS) F ON F.city = AR.city AND F.month_date = AR.month_date AND F.day = AR.day
            LEFT JOIN (SELECT city, date_trunc('month', day) month_date, AVG(revenue_start) forecast_r FROM BASE_DATA GROUP BY city, date_trunc('month', day)) F2 ON F2.city = AR.city AND F2.month_date = AR.month_date

        ORDER BY AR.city ASC, AR.month_date ASC, AR.day ASC
    )

    ,PRE_DELTAS AS (
        SELECT 
            city, month_date, day, comset, 
            SUM(revenue) OVER (PARTITION BY city, month_date ORDER BY month_date ASC, day ASC) revenue,
            SUM(forecast_r) OVER (PARTITION BY city, month_date ORDER BY month_date ASC, day ASC) forecast_r

        FROM PRE_FINAL
    )

    SELECT 
    city, month_date, day, comset, 
    revenue, forecast_r, (revenue - forecast_r) revenue_delta, 
    (revenue - forecast_r) / forecast_r pct_revenue_delta
    
    FROM PRE_DELTAS
"""

spark.sql(revenue_by_city).write.mode("overwrite").saveAsTable("silver.finance.revenue_by_city")

### 4. GM1 - by City

In [0]:
gm1_by_city = """
WITH Revenue AS (
        SELECT * FROM silver.finance.revenue_by_city
    )

    ,occupied AS (
        SELECT city, month_date, day, comset, occupied_nights FROM silver.finance.occupation_by_city
    )

    ,check_out AS (
        SELECT 
            trunc(check_out, 'month') month_date, city,
            CASE 
                WHEN date_trunc('month', sell_date) = date_trunc('month', NOW()) THEN CAST(date_trunc('day', sell_date) AS DATE)
                WHEN date_trunc('month', sell_date) < date_trunc('month', NOW()) THEN CAST(date_trunc('month', NOW()) - INTERVAL '1 DAY' AS DATE)
            END AS day,
            COUNT(check_out) check_out
        
        FROM silver.finance.new_sales_and_source
        
        WHERE date_trunc('month', sell_date) <= date_trunc('month', NOW())
            AND trunc(check_out, 'month') BETWEEN date_trunc('month', NOW())
            AND date_trunc('month', NOW()) + INTERVAL '3 month'

        GROUP BY city, trunc(check_out, 'month'), CASE WHEN date_trunc('month', sell_date) = date_trunc('month', NOW()) THEN CAST(date_trunc('day', sell_date) AS DATE)
                                                    WHEN date_trunc('month', sell_date) < date_trunc('month', NOW()) THEN CAST(date_trunc('month', NOW()) - INTERVAL '1 DAY' AS DATE)
                                                END
    )

    ,Volume AS (
        SELECT * FROM silver.finance.volume_database
    )

    ,Ttl_apts AS (
        SELECT SUM(guest_ready) apts, month_date, city FROM volume GROUP BY month_date, city
    )


    ---------------------------------------------------------------------------------------------------
    --------------------------------- UTILITIES FORECAST ----------------------------------------------
    ---------------------------------------------------------------------------------------------------

    ,Slopes_Utilities_1 AS (
        SELECT *, explode(split(Feature,'_')) feature_2 FROM silver.finance.cogs_forecast_slopes WHERE (Feature ILIKE '%utilities%' OR Feature = 'Booked')
    )

    ,Slopes_Utilities AS (
        SELECT Coefficient, feature_2 feature FROM Slopes_Utilities_1 WHERE feature_2 <> 'Utilities'
    )

    ,Occupied_Fcst AS (
        SELECT 
            o.day, o.city, o.month_date, o.comset, o.occupied_nights,
            CASE WHEN day = trunc(NOW(), 'month') -  INTERVAL 1 DAY THEN ((o.occupied_nights::DECIMAL * (uu.Coefficient)) + (uuu.Coefficient) + COALESCE((u.Coefficient),0))
                ELSE (o.occupied_nights::DECIMAL * (uu.Coefficient))
            END AS amount

        FROM occupied o 
            LEFT JOIN Slopes_Utilities u ON u.feature = o.city
            LEFT JOIN Ttl_apts t ON t.city = o.city AND t.month_date = o.month_date
            CROSS JOIN Slopes_Utilities uu 
            CROSS JOIN Slopes_Utilities uuu  
        
        WHERE uu.feature = 'Booked' AND uuu.feature = 'Intercept'
    )


    ---------------------------------------------------------------------------------------------------
    ---------------------------------- CLEANING FORECAST ----------------------------------------------
    ---------------------------------------------------------------------------------------------------

    ,Slopes_FC_1 AS (
        SELECT *, explode(split(Feature,'_')) feature_2 FROM silver.finance.cogs_forecast_slopes WHERE (Feature ILIKE '%fc%' OR Feature = 'Check_out')
    )

    ,Slopes_FC AS (
        SELECT Coefficient, feature_2 feature FROM Slopes_FC_1 WHERE feature_2 NOT IN ('FC', 'out')
    )

    ,Cleaning_Fcst AS (
        SELECT
            c.day, c.city, c.month_date, c.check_out,
            CASE WHEN day = trunc(NOW(), 'month') -  INTERVAL 1 DAY THEN ((c.check_out::DECIMAL * (ss.Coefficient)) + (sss.Coefficient) + COALESCE((s.Coefficient),0))
                ELSE (c.check_out::DECIMAL * (ss.Coefficient))
            END AS amount

        FROM Check_Out c 
            LEFT JOIN Slopes_FC s ON s.feature = c.city
            LEFT JOIN Ttl_apts t ON t.city = c.city AND t.month_date = c.month_date
            CROSS JOIN Slopes_FC ss 
            CROSS JOIN Slopes_FC sss  
        
        WHERE ss.feature = 'Check' AND sss.feature = 'Intercept'
    )


    ---------------------------------------------------------------------------------------------------
    -------------------------------------- RENT COGS --------------------------------------------------
    ---------------------------------------------------------------------------------------------------

    ,Additional_costs_2 AS (
        SELECT ams_contract_terms.apartment_id, city, JSON_TUPLE(RENT_INFO, 'additional_costs_amount') AS additional_costs_amount
        FROM silver.backoffice.ams_contract_terms LEFT JOIN silver.Backoffice.apartments ON apartments.id = ams_contract_terms.apartment_id
    )

    ,Backoffice_3 AS (
        SELECT SUM(landlord_daily_rent)::decimal as amount, trunc(day, 'month') as month, city, 
        landlord.apartment_id, landlord.end_of_grace_period
        FROM silver.revenue.landlord LEFT JOIN silver.Backoffice.apartments ON apartments.id = landlord.apartment_id
        WHERE trunc(day, 'month') >= trunc(current_date, 'month') - INTERVAL '1 MONTH'
        GROUP BY trunc(day, 'month'), city, landlord.apartment_id, landlord.end_of_grace_period
    )

    ,Backoffice_2 AS (
        SELECT 
            Backoffice_3.apartment_id, Backoffice_3.city, Backoffice_3.amount, Backoffice_3.month, 
            CASE 
                WHEN Backoffice_3.month = trunc(end_of_grace_period, 'month') 
                    THEN (COALESCE(CAST(additional_costs_amount AS DECIMAL),0) / EXTRACT(DAY FROM trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY)) * (EXTRACT(DAY FROM trunc(month, 'month') + INTERVAL 1 MONTH - INTERVAL 1 DAY) - EXTRACT(DAY FROM end_of_grace_period))
                WHEN Backoffice_3.amount = 0 THEN 0
                ELSE COALESCE(CAST(additional_costs_amount AS DECIMAL),0)
            END AS additional_costs_amount
        FROM Backoffice_3
            JOIN Additional_costs_2 ON Backoffice_3.apartment_id = Additional_costs_2.apartment_id
    )

    ,Backoffice AS (
        SELECT (SUM(amount) + SUM(additional_costs_amount))*-1 amount, city, month month_date
        FROM Backoffice_2 
        GROUP BY city, month
    )

    ,COGS_FORECAST AS (
        SELECT city, db_month month_date, SUM(amount) value 
        FROM silver.finance.best_case_budget_finance 
        WHERE account ILIKE ('%COGS%')
        GROUP BY city, db_month
    )

    ,PRE_FINAL AS (
        SELECT  
            R.city, R.month_date, R.day, R.comset, R.revenue, R.forecast_r,
            (SUM(COALESCE(CF.amount,0)) OVER (PARTITION BY R.city, R.month_date ORDER BY R.month_date ASC, R.day ASC) + 
            (SUM(COALESCE(OF.amount,0)) OVER (PARTITION BY R.city, R.month_date ORDER BY R.month_date ASC, R.day ASC) + B.amount)) cogs,
            C.value forecast_c

        FROM Revenue R
            LEFT JOIN Cleaning_Fcst CF ON R.city = CF.city AND R.month_date = CF.month_date AND R.day = CF.day
            LEFT JOIN Occupied_Fcst OF ON OF.city = R.city AND OF.month_date = R.month_date AND OF.day = R.day
            LEFT JOIN Backoffice B ON B.city = R.city AND B.month_date = R.month_date
            LEFT JOIN COGS_FORECAST C ON C.city = R.city AND TO_DATE(C.month_date, 'dd/MM/yyyy') = TO_DATE(R.month_date, 'dd/MM/yyyy')
    )


    SELECT 
        city, month_date, day, comset, 
        (revenue + cogs) / revenue gm1, (forecast_r + forecast_c) / forecast_r forecast_gm1,
        ((revenue + cogs) / revenue) - ((forecast_r + forecast_c) / forecast_r) gm1_delta,
        revenue, forecast_r, cogs, forecast_c

    FROM PRE_FINAL ORDER BY city ASC, month_date ASC, day ASC
"""

spark.sql(gm1_by_city).write.mode("overwrite").saveAsTable("silver.finance.gm1_by_city")